En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [2]:
file_path = "farmers-protest-tweets-2021-2-4.json"

# PREGUNTA 1

1. Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. Debe incluir las siguientes funciones:
```python
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
Returns: 
[(datetime.date(1999, 11, 15), "LATAM321"), (datetime.date(1999, 7, 15), "LATAM_CHI"), ...]
```
## PREGUNTA 1 - TIME (q1_time)
Para desarrollar la solución a este problema se utilizará una solución cloud basada en Google Cloud Platform. La función contendrá el proceso de ETL para llevar los datos desde el archivo json local hacia Google Cloud Storage y posteriormente modelar esa data en una tabla de bigquery que permita realizar consultas de manera rápida y eficiente.
### ETL
El proceso de extracción, transformación y carga de los archivos en GCP permite llevar los archivos a una plataforma de performance rápida y efectiva.
#### Credenciales
Este desarrollo se realizará utilizando Google Cloud, por lo que se crea un proyecto en GCP llamado "project-latam-challenge". En este proyecto se crea una service account llamada "sa-etl-latam-challenge" que será utilizada para realizar la carga de datos.
```python
import os

# Ruta a archivo de credenciales JSON de Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../credentials/project-latam-challenge-749ce1a96052.json"
```
#### Carga en Dataframe
Se carga en un dataframe el [archivo](https://drive.google.com/file/d/1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis/view?usp=sharing) json declarado como parte del challenge.
```python
    # Leer el archivo CSV en un DataFrame
    df = pd.read_json(file_path,lines=True)
```
#### Carga de dataframe en GCP
##### Carga de archivo en Google Cloud Storage (GCS)
Usando Google Cloud SDK se crea un bucket llamado 'bucket-project-latam-challenge-q1-time' en el proyecto 'project-latam-challenge' y se carga el archivo directamente a dicho bucket.
```python
    # Se define función que permite cargar un archivo en un bucket de Google Cloud Storage
    def create_bucket_and_upload_file(project_id, bucket_name, file_path, destination_blob_name):
        # Inicializar el cliente de almacenamiento
        storage_client = storage.Client(project=project_id)
        
        # Verificar si el bucket ya existe
        bucket = storage_client.bucket(bucket_name)
        if not bucket.exists():
            # Crear un nuevo bucket con la ubicación especificada
            new_bucket = storage_client.create_bucket(bucket, location="US")
            print(f'Bucket {bucket_name} created in location US.')
        else:
            print(f'Bucket {bucket_name} already exists.')
            new_bucket = bucket
        
        # Subir el archivo al bucket
        blob = new_bucket.blob(destination_blob_name)
        blob.upload_from_filename(file_path)
        print(f'File {file_path} uploaded to {bucket_name}/{destination_blob_name}.')
```
##### Configuración de carga
```python
from google.cloud import bigquery

# Parámetros
project_id = 'project-latam-challenge'
dataset_id = 'twitter_data'
table_id = 'farmers_protest_tweets_2021'
```
##### Declaración de schema
Se declara explicitamente la estructura del esquema con la data correspondiente, esto permitirá evitar errores en los tipos de dato al cargar la data en Bigquery.
Por motivos de claridad al momento de leer el markdown, se omite el schema que se puede encontrar en el archivo q1_time.py .
##### Proceso de creación de tabla en bigquery
```python
    # Se define función que crea tabla de bigquery a partir de archivo almacenado en GCS
    def load_data_from_gcs_to_bigquery(uri, table_id):
        # Inicializa el cliente de BigQuery
        client = bigquery.Client()

        job_config = bigquery.LoadJobConfig(
            schema=schema,
            source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
            max_bad_records=0,  # No permitir registros malos antes de fallar
            time_partitioning=bigquery.TimePartitioning(
                type_=bigquery.TimePartitioningType.DAY,
                field="date"  # Campo de partición
            )
        )

        load_job = client.load_table_from_uri(
            uri, table_id, job_config=job_config
        )

        print(f'Starting job {load_job.job_id}')
        load_job.result()
        print(f'Job finished.')

        destination_table = client.get_table(table_id)
        print(f'Loaded {destination_table.num_rows} rows.')
```
##### Análisis
Se define función que permite ejecutar la consulta en bigquery.

```python
    # Se define función para ejecutar una consulta en BigQuery
    def run_bigquery_query(query: str):
        client = get_bigquery_client()
        query_job = client.query(query)
        results = query_job.result()
        return results
```
Se define la query SQL que permite ejecutar la consulta en bigquery.
```python
    query = """
        WITH TEMP_DATA_001 AS 
        (
        -- Se crea la tabla temporal sólo con los campos necesarios para realizar ambos cálculos, tanto por fecha como por usuario
        SELECT DATE(date) AS fecha,
            id,
            user.username AS username
        FROM `project-latam-challenge.twitter_data.farmers_protest_tweets_2021` 
        WHERE DATE(date) IS NOT NULL
        ), TEMP_DATE_001 AS 
        (
        -- Se realiza el cálculo de la cantidad de tweets por fecha
        SELECT fecha,
            COUNT(DISTINCT id) AS tweet_qty
        FROM TEMP_DATA_001
        GROUP BY fecha
        ), TEMP_DATE_002 AS 
        (
        -- Se realiza el ranking de tweets ordenados descendentemente por cantidad de tweets y aleatoriamente
        SELECT A.*,
            RANK() OVER(ORDER BY tweet_qty DESC, RAND()) AS ranking_tweet 
        FROM TEMP_DATE_001 A
        ), TEMP_USER_001 AS 
        (
        -- Se realiza el cálculo de la cantidad de tweets por usuario y fecha
        SELECT fecha,
            username,
            COUNT(DISTINCT id) AS tweet_qty
        FROM TEMP_DATA_001
        GROUP BY fecha, username
        ), TEMP_USER_002 AS 
        -- Se realiza el ranking de usuarios por fecha, ordenados descendentemente por cantidad de tweets y aleatoriamente
        (
        SELECT A.*,
            RANK() OVER(PARTITION BY fecha ORDER BY tweet_qty DESC, RAND()) AS ranking_user
        FROM TEMP_USER_001 A
        ), TEMP_USER_003 AS 
        (
        -- Se seleccionan los usuarios con ranking 1, es decir, aquellos que más tweets hicieron por día
        SELECT A.*
        FROM TEMP_USER_002 A
        WHERE ranking_user=1
        )
        -- Se filtra la tabla temporal de tweets por fecha, seleccionando sólo los días que se encuentran en el top 10
        SELECT A.fecha,
            B.username
        FROM TEMP_DATE_002 A
        LEFT JOIN TEMP_USER_003 B
        ON A.fecha=B.fecha 
        WHERE A.ranking_tweet<=10
        ORDER BY ranking_tweet
        """
    
    results = run_bigquery_query(query)
    return [(row.fecha, row.username) for row in results]
```

### Evaluación

In [2]:
import os
import time
from memory_profiler import memory_usage
from q1_time import q1_time
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../credentials/project-latam-challenge-749ce1a96052.json"

# Función para medir el tiempo y la memoria de q1_time
def measure_q1_time(file_path):
    start_time = time.time()
    mem_usage, result = memory_usage((q1_time, (file_path,)), retval=True, max_usage=True)
    end_time = time.time()
    duration = end_time - start_time
    return result, duration, mem_usage

if __name__ == "__main__":
    print("Iniciando proceso q1_time...")
    top_10_dates, duration, mem_usage = measure_q1_time(file_path)
    print("Resultados:")
    print(top_10_dates)
    print(f"Tiempo de ejecución: {duration} segundos")
    print(f"Uso máximo de memoria: {mem_usage} MiB")

Iniciando proceso q1_time...
Leyendo archivo JSON
Creando bucket y subiendo archivo
Bucket bucket-project-latam-challenge-q1-time created in location US.
File farmers-protest-tweets-2021-2-4.json uploaded to bucket-project-latam-challenge-q1-time/farmers-protest-tweets-2021-2-4.json.
Cargando datos a BigQuery
Starting job 22e9f6c9-c732-4439-b44f-a688290fc0ea
Job finished.
Loaded 117407 rows.
Ejecutando query
Resultados:
[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]
Tiempo de ejecución: 59.299320936203 segundos
Uso máximo de memoria: 2244.50390625 MiB


## PREGUNTA 1 - MEMORY
Se propone generar un proceso análogo al anteriormente realizado para la versión time, pero con una carga parcelada de forma que utilice menos memoria local y se ejecute con menor necesidad de recursos.

2. Los top 10 emojis más usados con su respectivo conteo. Debe incluir las siguientes funciones:
```python
def q2_time(file_path: str) -> List[Tuple[str, int]]:
```
```python
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
```
```python
Returns: 
[("✈️", 6856), ("❤️", 5876), ...]
```
## PREGUNTA 2 - TIME (q2_time)
Para desarrollar la solución a este problema se utilizará una solución cloud basada en Google Cloud Platform. La función contendrá el proceso de ETL para llevar los datos desde el archivo json local hacia Google Cloud Storage y posteriormente modelar esa data en una tabla de bigquery que permita realizar consultas de manera rápida y eficiente.
### ETL
El proceso de extracción, transformación y carga de los archivos en GCP permite llevar los archivos a una plataforma de performance rápida y efectiva.
#### Credenciales
Este desarrollo se realizará utilizando Google Cloud, por lo que se crea un proyecto en GCP llamado "project-latam-challenge". En este proyecto se crea una service account llamada "sa-etl-latam-challenge" que será utilizada para realizar la carga de datos.
```python
import os

# Ruta a archivo de credenciales JSON de Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../credentials/project-latam-challenge-749ce1a96052.json"
```
#### Carga en Dataframe
Se carga en un dataframe el [archivo](https://drive.google.com/file/d/1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis/view?usp=sharing) json declarado como parte del challenge.
```python
    # Leer el archivo CSV en un DataFrame
    df = pd.read_json(file_path,lines=True)
```
#### Carga de dataframe en GCP
##### Carga de archivo en Google Cloud Storage (GCS)
Usando Google Cloud SDK se crea un bucket llamado 'bucket-project-latam-challenge-q1-time' en el proyecto 'project-latam-challenge' y se carga el archivo directamente a dicho bucket.
```python
    # Se define función que permite cargar un archivo en un bucket de Google Cloud Storage
    def create_bucket_and_upload_file(project_id, bucket_name, file_path, destination_blob_name):
        # Inicializar el cliente de almacenamiento
        storage_client = storage.Client(project=project_id)
        
        # Verificar si el bucket ya existe
        bucket = storage_client.bucket(bucket_name)
        if not bucket.exists():
            # Crear un nuevo bucket con la ubicación especificada
            new_bucket = storage_client.create_bucket(bucket, location="US")
            print(f'Bucket {bucket_name} created in location US.')
        else:
            print(f'Bucket {bucket_name} already exists.')
            new_bucket = bucket
        
        # Subir el archivo al bucket
        blob = new_bucket.blob(destination_blob_name)
        blob.upload_from_filename(file_path)
        print(f'File {file_path} uploaded to {bucket_name}/{destination_blob_name}.')
```
##### Configuración de carga
```python
from google.cloud import bigquery

# Parámetros
project_id = 'project-latam-challenge'
dataset_id = 'twitter_data'
table_id = 'farmers_protest_tweets_2021'
```
##### Declaración de schema
Se declara explicitamente la estructura del esquema con la data correspondiente, esto permitirá evitar errores en los tipos de dato al cargar la data en Bigquery.
Por motivos de claridad al momento de leer el markdown, se omite el schema que se puede encontrar en el archivo q1_time.py .
##### Proceso de creación de tabla en bigquery
```python
    # Se define función que crea tabla de bigquery a partir de archivo almacenado en GCS
    def load_data_from_gcs_to_bigquery(uri, table_id):
        # Inicializa el cliente de BigQuery
        client = bigquery.Client()

        job_config = bigquery.LoadJobConfig(
            schema=schema,
            source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
            max_bad_records=0,  # No permitir registros malos antes de fallar
            time_partitioning=bigquery.TimePartitioning(
                type_=bigquery.TimePartitioningType.DAY,
                field="date"  # Campo de partición
            )
        )

        load_job = client.load_table_from_uri(
            uri, table_id, job_config=job_config
        )

        print(f'Starting job {load_job.job_id}')
        load_job.result()
        print(f'Job finished.')

        destination_table = client.get_table(table_id)
        print(f'Loaded {destination_table.num_rows} rows.')
```
##### Análisis
Se define función que permite ejecutar la consulta en bigquery.

```python
    # Se define función para ejecutar una consulta en BigQuery
    def run_bigquery_query(query: str):
        client = get_bigquery_client()
        query_job = client.query(query)
        results = query_job.result()
        return results
```
Se define la query SQL que permite ejecutar la consulta en bigquery.
La lista de emojis se obtiene de https://gist.github.com/bfeldman89/fb25ddb63bdaa6de6ab7ac946acde96f#file-emojis-csv .
```python
    query = """
   WITH EmojiExtractor AS (
    -- Separa todos los caracteres de cada tweet
    SELECT
        content,
        SPLIT(content, '') AS chars
    FROM
        `project-latam-challenge.twitter_data.farmers_protest_tweets_2021`
    ), EmojiCount AS (
    -- Calcula la frecuencia de cada caracter
    SELECT char,
        COUNT(*) AS frequency
    FROM EmojiExtractor
    LEFT JOIN UNNEST(chars) AS char
    WHERE
    --LISTA DE POSIBLES EMOJIS
        char IN (
"😀","😁","😂","🤣","😃","😄","😅","😆","😉","😊","😋","😎","😍","😘","😗","😙","😚","☺","🙂","🤗","🤩","🤔","🤨","😐","😑","😶","🙄","😏","😣","😥","😮","🤐","😯","😪","😫","😴","😌","😛","😜","😝","🤤","😒","😓","😔","😕","🙃","🤑","😲","☹","🙁","😖","😞","😟","😤","😢","😭","😦","😧","😨","😩","🤯","😬","😰","😱","😳","🤪","😵","😡","😠","🤬","😷","🤒","🤕","🤢","🤮","🤧","😇","🤠","🤡","🤥","🤫","🤭","🧐","🤓","😈","👿","👹","👺","💀","👻","👽","🤖","💩","😺","😸","😹","😻","😼","😽","🙀","😿","😾","👶","👦","👧","👨","👩","👴","👵","👨‍⚕️","👩‍⚕️","👨‍🎓","👩‍🎓","👨‍⚖️","👩‍⚖️","👨‍🌾","👩‍🌾","👨‍🍳","👩‍🍳","👨‍🔧","👩‍🔧","👨‍🏭","👩‍🏭","👨‍💼","👩‍💼","👨‍🔬","👩‍🔬","👨‍💻","👩‍💻","👨‍🎤","👩‍🎤","👨‍🎨","👩‍🎨","👨‍✈️","👩‍✈️","👨‍🚀","👩‍🚀","👨‍🚒","👩‍🚒","👮","👮‍♂️","👮‍♀️","🕵","🕵️‍♂️","🕵️‍♀️","💂","💂‍♂️","💂‍♀️","👷","👷‍♂️","👷‍♀️","🤴","👸","👳","👳‍♂️","👳‍♀️","👲","🧕","🧔","👱","👱‍♂️","👱‍♀️","🤵","👰","🤰","🤱","👼","🎅","🤶","🧙‍♀️","🧙‍♂️","🧚‍♀️","🧚‍♂️","🧛‍♀️","🧛‍♂️","🧜‍♀️","🧜‍♂️","🧝‍♀️","🧝‍♂️","🧞‍♀️","🧞‍♂️","🧟‍♀️","🧟‍♂️","🙍","🙍‍♂️","🙍‍♀️","🙎","🙎‍♂️","🙎‍♀️","🙅","🙅‍♂️","🙅‍♀️","🙆","🙆‍♂️","🙆‍♀️","💁","💁‍♂️","💁‍♀️","🙋","🙋‍♂️","🙋‍♀️","🙇","🙇‍♂️","🙇‍♀️","🤦","🤦‍♂️","🤦‍♀️","🤷","🤷‍♂️","🤷‍♀️","💆","💆‍♂️","💆‍♀️","💇","💇‍♂️","💇‍♀️","🚶","🚶‍♂️","🚶‍♀️","🏃","🏃‍♂️","🏃‍♀️","💃","🕺","👯","👯‍♂️","👯‍♀️","🧖‍♀️","🧖‍♂️","🕴","🗣","👤","👥","👫","👬","👭","💏","👨‍❤️‍💋‍👨","👩‍❤️‍💋‍👩","💑","👨‍❤️‍👨","👩‍❤️‍👩","👪","👨‍👩‍👦","👨‍👩‍👧","👨‍👩‍👧‍👦","👨‍👩‍👦‍👦","👨‍👩‍👧‍👧","👨‍👨‍👦","👨‍👨‍👧","👨‍👨‍👧‍👦","👨‍👨‍👦‍👦","👨‍👨‍👧‍👧","👩‍👩‍👦","👩‍👩‍👧","👩‍👩‍👧‍👦","👩‍👩‍👦‍👦","👩‍👩‍👧‍👧","👨‍👦","👨‍👦‍👦","👨‍👧","👨‍👧‍👦","👨‍👧‍👧","👩‍👦","👩‍👦‍👦","👩‍👧","👩‍👧‍👦","👩‍👧‍👧","🤳","💪","👈","👉","☝","👆","🖕","👇","✌","🤞","🖖","🤘","🖐","✋","👌","👍","👎","✊","👊","🤛","🤜","🤚","👋","🤟","✍","👏","👐","🙌","🤲","🙏","🤝","💅","👂","👃","👣","👀","👁","🧠","👅","👄","💋","👓","🕶","👔","👕","👖","🧣","🧤","🧥","🧦","👗","👘","👙","👚","👛","👜","👝","🎒","👞","👟","👠","👡","👢","👑","👒","🎩","🎓","🧢","⛑","💄","💍","🌂","☂","💼","break","🙈","🙉","🙊","💥","💦","💨","💫","🐵","🐒","🦍","🐶","🐕","🐩","🐺","🦊","🐱","🐈","🦁","🐯","🐅","🐆","🐴","🐎","🦄","🦓","🐮","🐂","🐃","🐄","🐷","🐖","🐗","🐽","🐏","🐑","🐐","🐪","🐫","🦒","🐘","🦏","🐭","🐁","🐀","🐹","🐰","🐇","🐿","🦔","🦇","🐻","🐨","🐼","🐾","🦃","🐔","🐓","🐣","🐤","🐥","🐦","🐧","🕊","🦅","🦆","🦉","🐸","🐊","🐢","🦎","🐍","🐲","🐉","🦕","🦖","🐳","🐋","🐬","🐟","🐠","🐡","🦈","🐙","🐚","🦀","🦐","🦑","🐌","🦋","🐛","🐜","🐝","🐞","🦗","🕷","🕸","🦂","💐","🌸","💮","🏵","🌹","🥀","🌺","🌻","🌼","🌷","🌱","🌲","🌳","🌴","🌵","🌾","🌿","☘","🍀","🍁","🍂","🍃","🍄","🌰","🌍","🌎","🌏","🌐","🌑","🌒","🌓","🌔","🌕","🌖","🌗","🌘","🌙","🌚","🌛","🌜","☀","🌝","🌞","⭐","🌟","🌠","☁","⛅","⛈","🌤","🌥","🌦","🌧","🌨","🌩","🌪","🌫","🌬","🌈","☂","☔","⚡","❄","☃","⛄","☄","🔥","💧","🌊","🎄","✨","🎋","🎍","break","🍇","🍈","🍉","🍊","🍋","🍌","🍍","🍎","🍏","🍐","🍑","🍒","🍓","🥝","🍅","🥥","🥑","🍆","🥔","🥕","🌽","🌶","🥒","🥦","🍄","🥜","🌰","🍞","🥐","🥖","🥨","🥞","🧀","🍖","🍗","🥩","🥓","🍔","🍟","🍕","🌭","🥪","🌮","🌯","🍳","🍲","🥣","🥗","🍿","🥫","🍱","🍘","🍙","🍚","🍛","🍜","🍝","🍠","🍢","🍣","🍤","🍥","🍡","🥟","🥠","🥡","🍦","🍧","🍨","🍩","🍪","🎂","🍰","🥧","🍫","🍬","🍭","🍮","🍯","🍼","🥛","☕","🍵","🍶","🍾","🍷","🍸","🍹","🍺","🍻","🥂","🥃","🥤","🥢","🍽","🍴","🥄","break","👾","🧗‍♀️","🧗‍♂️","🧘‍♀️","🧘‍♂️","🕴","🏇","⛷","🏂","🏌","🏌️‍♂️","🏌️‍♀️","🏄","🏄‍♂️","🏄‍♀️","🚣","🚣‍♂️","🚣‍♀️","🏊","🏊‍♂️","🏊‍♀️","⛹","⛹️‍♂️","⛹️‍♀️","🏋","🏋️‍♂️","🏋️‍♀️","🚴","🚴‍♂️","🚴‍♀️","🚵","🚵‍♂️","🚵‍♀️","🤸","🤸‍♂️","🤸‍♀️","🤼","🤼‍♂️","🤼‍♀️","🤽","🤽‍♂️","🤽‍♀️","🤾","🤾‍♂️","🤾‍♀️","🤹","🤹‍♂️","🤹‍♀️","🎪","🎗","🎟","🎫","🎖","🏆","🏅","🥇","🥈","🥉","⚽","⚾","🏀","🏐","🏈","🏉","🎾","🎳","🏏","🏑","🏒","🏓","🏸","🥊","🥋","⛳","⛸","🎣","🎽","🎿","🛷","🥌","🎯","🎱","🎮","🎰","🎲","🎭","🎨","🎼","🎤","🎧","🎷","🎸","🎹","🎺","🎻","🥁","🎬","🏹","break","🚣","🏎","🏍","🗾","🏔","⛰","🌋","🗻","🏕","🏖","🏜","🏝","🏞","🏟","🏛","🏗","🏘","🏚","🏠","🏡","🏢","🏣","🏤","🏥","🏦","🏨","🏩","🏪","🏫","🏬","🏭","🏯","🏰","💒","🗼","🗽","⛪","🕌","🕍","⛩","🕋","⛲","⛺","🌁","🌃","🏙","🌄","🌅","🌆","🌇","🌉","🌌","🎠","🎡","🎢","🚂","🚃","🚄","🚅","🚆","🚇","🚈","🚉","🚊","🚝","🚞","🚋","🚌","🚍","🚎","🚐","🚑","🚒","🚓","🚔","🚕","🚖","🚗","🚘","🚚","🚛","🚜","🚲","🛴","🛵","🚏","🛤","⛽","🚨","🚥","🚦","🚧","⚓","⛵","🚤","🛳","⛴","🛥","🚢","✈","🛩","🛫","🛬","💺","🚁","🚟","🚠","🚡","🛰","🚀","🛸","🌠","⛱","🎆","🎇","🎑","💴","💵","💶","💷","🗿","🛂","🛃","🛄","🛅","break","☠","🛀","🛌","💌","💣","🕳","🛍","📿","💎","🔪","🏺","🗺","💈","🛢","🛎","⌛","⏳","⌚","⏰","⏱","⏲","🕰","🌡","⛱","🎈","🎉","🎊","🎎","🎏","🎐","🎀","🎁","🔮","🕹","🖼","📯","🎙","🎚","🎛","📻","📱","📲","☎","📞","📟","📠","🔋","🔌","💻","🖥","🖨","⌨","🖱","🖲","💽","💾","💿","📀","🎥","🎞","📽","📺","📷","📸","📹","📼","🔍","🔎","🕯","💡","🔦","🏮","📔","📕","📖","📗","📘","📙","📚","📓","📃","📜","📄","📰","🗞","📑","🔖","🏷","💰","💴","💵","💶","💷","💸","💳","✉","📧","📨","📩","📤","📥","📦","📫","📪","📬","📭","📮","🗳","✏","✒","🖋","🖊","🖌","🖍","📝","📁","📂","🗂","📅","📆","🗒","🗓","📇","📈","📉","📊","📋","📌","📍","📎","🖇","📏","📐","✂","🗃","🗄","🗑","🔒","🔓","🔏","🔐","🔑","🗝","🔨","⛏","⚒","🛠","🗡","⚔","🔫","🛡","🔧","🔩","⚙","🗜","⚖","🔗","⛓","⚗","🔬","🔭","📡","💉","💊","🚪","🛏","🛋","🚽","🚿","🛁","🚬","⚰","⚱","🗿","🚰","break","👁️‍🗨️","💘","❤","💓","💔","💕","💖","💗","💙","💚","💛","🧡","💜","🖤","💝","💞","💟","❣","💤","💢","💬","🗯","💭","💮","♨","💈","🛑","🕛","🕧","🕐","🕜","🕑","🕝","🕒","🕞","🕓","🕟","🕔","🕠","🕕","🕡","🕖","🕢","🕗","🕣","🕘","🕤","🕙","🕥","🕚","🕦","🌀","♠","♥","♦","♣","🃏","🀄","🎴","🔇","🔈","🔉","🔊","📢","📣","📯","🔔","🔕","🎵","🎶","🏧","🚮","🚰","♿","🚹","🚺","🚻","🚼","🚾","⚠","🚸","⛔","🚫","🚳","🚭","🚯","🚱","🚷","🔞","☢","☣","⬆","↗","➡","↘","⬇","↙","⬅","↖","↕","↔","↩","↪","⤴","⤵","🔃","🔄","🔙","🔚","🔛","🔜","🔝","🛐","⚛","🕉","✡","☸","☯","✝","☦","☪","☮","🕎","🔯","♈","♉","♊","♋","♌","♍","♎","♏","♐","♑","♒","♓","⛎","🔀","🔁","🔂","▶","⏩","◀","⏪","🔼","⏫","🔽","⏬","⏹","⏏","🎦","🔅","🔆","📶","📳","📴","♻","🔱","📛","🔰","⭕","✅","☑","✔","✖","❌","❎","➕","➖","➗","➰","➿","〽","✳","✴","❇","‼","⁉","❓","❔","❕","❗","©","®","™","️⃣","0️⃣","1️⃣","2️⃣","3️⃣","4️⃣","5️⃣","6️⃣","7️⃣","8️⃣","9️⃣","🔟","💯","🔠","🔡","🔢","🔣","🔤","🅰","🆎","🅱","🆑","🆒","🆓","ℹ","🆔","Ⓜ","🆕","🆖","🅾","🆗","🅿","🆘","🆙","🆚","🈁","🈂","🈷","🈶","🈯","🉐","🈹","🈚","🈲","🉑","🈸","🈴","🈳","㊗","㊙","🈺","🈵","▪","▫","◻","◼","◽","◾","⬛","⬜","🔶","🔷","🔸","🔹","🔺","🔻","💠","🔲","🔳","⚪","⚫","🔴","🔵","break","🏁","🚩","🎌","🏴","🏳","🏳️‍🌈","🏴‍☠️","🇦🇨","🇦🇩","🇦🇪","🇦🇫","🇦🇬","🇦🇮","🇦🇱","🇦🇲","🇦🇴","🇦🇶","🇦🇷","🇦🇸","🇦🇹","🇦🇺","🇦🇼","🇦🇽","🇦🇿","🇧🇦","🇧🇧","🇧🇩","🇧🇪","🇧🇫","🇧🇬","🇧🇭","🇧🇮","🇧🇯","🇧🇱","🇧🇲","🇧🇳","🇧🇴","🇧🇶","🇧🇷","🇧🇸","🇧🇹","🇧🇻","🇧🇼","🇧🇾","🇧🇿","🇨🇦","🇨🇨","🇨🇩","🇨🇫","🇨🇬","🇨🇭","🇨🇮","🇨🇰","🇨🇱","🇨🇲","🇨🇳","🇨🇴","🇨🇵","🇨🇷","🇨🇺","🇨🇻","🇨🇼","🇨🇽","🇨🇾","🇨🇿","🇩🇪","🇩🇬","🇩🇯","🇩🇰","🇩🇲","🇩🇴","🇩🇿","🇪🇦","🇪🇨","🇪🇪","🇪🇬","🇪🇭","🇪🇷","🇪🇸","🇪🇹","🇪🇺","🇫🇮","🇫🇯","🇫🇰","🇫🇲","🇫🇴","🇫🇷","🇬🇦","🇬🇧","🇬🇩","🇬🇪","🇬🇫","🇬🇬","🇬🇭","🇬🇮","🇬🇱","🇬🇲","🇬🇳","🇬🇵","🇬🇶","🇬🇷","🇬🇸","🇬🇹","🇬🇺","🇬🇼","🇬🇾","🇭🇰","🇭🇲","🇭🇳","🇭🇷","🇭🇹","🇭🇺","🇮🇨","🇮🇩","🇮🇪","🇮🇱","🇮🇲","🇮🇳","🇮🇴","🇮🇶","🇮🇷","🇮🇸","🇮🇹","🇯🇪","🇯🇲","🇯🇴","🇯🇵","🇰🇪","🇰🇬","🇰🇭","🇰🇮","🇰🇲","🇰🇳","🇰🇵","🇰🇷","🇰🇼","🇰🇾","🇰🇿","🇱🇦","🇱🇧","🇱🇨","🇱🇮","🇱🇰","🇱🇷","🇱🇸","🇱🇹","🇱🇺","🇱🇻","🇱🇾","🇲🇦","🇲🇨","🇲🇩","🇲🇪","🇲🇫","🇲🇬","🇲🇭","🇲🇰","🇲🇱","🇲🇲","🇲🇳","🇲🇴","🇲🇵","🇲🇶","🇲🇷","🇲🇸","🇲🇹","🇲🇺","🇲🇻","🇲🇼","🇲🇽","🇲🇾","🇲🇿","🇳🇦","🇳🇨","🇳🇪","🇳🇫","🇳🇬","🇳🇮","🇳🇱","🇳🇴","🇳🇵","🇳🇷","🇳🇺","🇳🇿","🇴🇲","🇵🇦","🇵🇪","🇵🇫","🇵🇬","🇵🇭","🇵🇰","🇵🇱","🇵🇲","🇵🇳","🇵🇷","🇵🇸","🇵🇹","🇵🇼","🇵🇾","🇶🇦","🇷🇪","🇷🇴","🇷🇸","🇷🇺","🇷🇼","🇸🇦","🇸🇧","🇸🇨","🇸🇩","🇸🇪","🇸🇬","🇸🇭","🇸🇮","🇸🇯","🇸🇰","🇸🇱","🇸🇲","🇸🇳","🇸🇴","🇸🇷","🇸🇸","🇸🇹","🇸🇻","🇸🇽","🇸🇾","🇸🇿","🇹🇦","🇹🇨","🇹🇩","🇹🇫","🇹🇬","🇹🇭","🇹🇯","🇹🇰","🇹🇱","🇹🇲","🇹🇳","🇹🇴","🇹🇷","🇹🇹","🇹🇻","🇹🇼","🇹🇿","🇺🇦","🇺🇬","🇺🇲","🇺🇳","🇺🇸","🇺🇾","🇺🇿","🇻🇦","🇻🇨","🇻🇪","🇻🇬","🇻🇮","🇻🇳","🇻🇺","🇼🇫","🇼🇸","🇽🇰","🇾🇪","🇾🇹","🇿🇦","🇿🇲","🇿🇼","🏴󠁧󠁢󠁥󠁮󠁧󠁿","🏴󠁧󠁢󠁳󠁣󠁴󠁿","🏴󠁧󠁢󠁷󠁬󠁳󠁿"
        )
    GROUP BY char
    )
    SELECT char AS emoji,
    frequency
    FROM EmojiCount
    ORDER BY frequency DESC
    LIMIT 10
        """
    
    results = run_bigquery_query(query)
    return [(row.fecha, row.username) for row in results]
```

In [3]:
import os
import time
from memory_profiler import memory_usage
from q2_time import q2_time
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../credentials/project-latam-challenge-749ce1a96052.json"

# Función para medir el tiempo y la memoria de q2_time
def measure_q1_time(file_path):
    start_time = time.time()
    mem_usage, result = memory_usage((q2_time, (file_path,)), retval=True, max_usage=True)
    end_time = time.time()
    duration = end_time - start_time
    return result, duration, mem_usage

if __name__ == "__main__":
    print("Iniciando proceso q1_time...")
    top_10_emojis, duration, mem_usage = measure_q1_time(file_path)
    print("Resultados:")
    print(top_10_emojis)
    print(f"Tiempo de ejecución: {duration} segundos")
    print(f"Uso máximo de memoria: {mem_usage} MiB")

Iniciando proceso q1_time...
Leyendo archivo JSON
Creando bucket y subiendo archivo
Bucket bucket-project-latam-challenge-q2-time created in location US.
File farmers-protest-tweets-2021-2-4.json uploaded to bucket-project-latam-challenge-q2-time/farmers-protest-tweets-2021-2-4.json.
Cargando datos a BigQuery
Starting job 1333d214-6ca2-451e-b71b-4424be85fa81
Job finished.
Loaded 234814 rows.
Ejecutando query
Resultados:
[('✊', 4822), ('❤', 3558), ('☮', 632), ('✌', 548), ('‼', 222), ('✍', 206), ('♥', 162), ('⛳', 136), ('✅', 114), ('➡', 112)]
Tiempo de ejecución: 51.47749996185303 segundos
Uso máximo de memoria: 2133.34765625 MiB


## PREGUNTA 2 - MEMORY
Se propone generar un proceso análogo al anteriormente realizado para la versión time, pero con una carga parcelada de forma que utilice menos memoria local y se ejecute con menor necesidad de recursos.

3. El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos. Debe incluir las siguientes funciones:
```python
def q3_time(file_path: str) -> List[Tuple[str, int]]:
```
```python
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
```
```python
Returns: 
[("LATAM321", 387), ("LATAM_CHI", 129), ...]
```
​
## PREGUNTA 3 - TIME (q3_time)
Para desarrollar la solución a este problema se utilizará una solución cloud basada en Google Cloud Platform. La función contendrá el proceso de ETL para llevar los datos desde el archivo json local hacia Google Cloud Storage y posteriormente modelar esa data en una tabla de bigquery que permita realizar consultas de manera rápida y eficiente.
### ETL
El proceso de extracción, transformación y carga de los archivos en GCP permite llevar los archivos a una plataforma de performance rápida y efectiva.
#### Credenciales
Este desarrollo se realizará utilizando Google Cloud, por lo que se crea un proyecto en GCP llamado "project-latam-challenge". En este proyecto se crea una service account llamada "sa-etl-latam-challenge" que será utilizada para realizar la carga de datos.
```python
import os

# Ruta a archivo de credenciales JSON de Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../credentials/project-latam-challenge-749ce1a96052.json"
```
#### Carga en Dataframe
Se carga en un dataframe el [archivo](https://drive.google.com/file/d/1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis/view?usp=sharing) json declarado como parte del challenge.
```python
    # Leer el archivo CSV en un DataFrame
    df = pd.read_json(file_path,lines=True)
```
#### Carga de dataframe en GCP
##### Carga de archivo en Google Cloud Storage (GCS)
Usando Google Cloud SDK se crea un bucket llamado 'bucket-project-latam-challenge-q1-time' en el proyecto 'project-latam-challenge' y se carga el archivo directamente a dicho bucket.
```python
    # Se define función que permite cargar un archivo en un bucket de Google Cloud Storage
    def create_bucket_and_upload_file(project_id, bucket_name, file_path, destination_blob_name):
        # Inicializar el cliente de almacenamiento
        storage_client = storage.Client(project=project_id)
        
        # Verificar si el bucket ya existe
        bucket = storage_client.bucket(bucket_name)
        if not bucket.exists():
            # Crear un nuevo bucket con la ubicación especificada
            new_bucket = storage_client.create_bucket(bucket, location="US")
            print(f'Bucket {bucket_name} created in location US.')
        else:
            print(f'Bucket {bucket_name} already exists.')
            new_bucket = bucket
        
        # Subir el archivo al bucket
        blob = new_bucket.blob(destination_blob_name)
        blob.upload_from_filename(file_path)
        print(f'File {file_path} uploaded to {bucket_name}/{destination_blob_name}.')
```
##### Configuración de carga
```python
from google.cloud import bigquery

# Parámetros
project_id = 'project-latam-challenge'
dataset_id = 'twitter_data'
table_id = 'farmers_protest_tweets_2021'
```
##### Declaración de schema
Se declara explicitamente la estructura del esquema con la data correspondiente, esto permitirá evitar errores en los tipos de dato al cargar la data en Bigquery.
Por motivos de claridad al momento de leer el markdown, se omite el schema que se puede encontrar en el archivo q1_time.py .
##### Proceso de creación de tabla en bigquery
```python
    # Se define función que crea tabla de bigquery a partir de archivo almacenado en GCS
    def load_data_from_gcs_to_bigquery(uri, table_id):
        # Inicializa el cliente de BigQuery
        client = bigquery.Client()

        job_config = bigquery.LoadJobConfig(
            schema=schema,
            source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
            max_bad_records=0,  # No permitir registros malos antes de fallar
            time_partitioning=bigquery.TimePartitioning(
                type_=bigquery.TimePartitioningType.DAY,
                field="date"  # Campo de partición
            )
        )

        load_job = client.load_table_from_uri(
            uri, table_id, job_config=job_config
        )

        print(f'Starting job {load_job.job_id}')
        load_job.result()
        print(f'Job finished.')

        destination_table = client.get_table(table_id)
        print(f'Loaded {destination_table.num_rows} rows.')
```
##### Análisis
Se define función que permite ejecutar la consulta en bigquery.

```python
    # Se define función para ejecutar una consulta en BigQuery
    def run_bigquery_query(query: str):
        client = get_bigquery_client()
        query_job = client.query(query)
        results = query_job.result()
        return results
```
Se define la query SQL que permite ejecutar la consulta en bigquery.
La lista de emojis se obtiene de https://gist.github.com/bfeldman89/fb25ddb63bdaa6de6ab7ac946acde96f#file-emojis-csv .
```python
    query = """
        WITH TEMP_DATA_001 AS 
        (
        SELECT mentionedUsers.username AS username
            -- quotedTweet.mentionedUsers.username,
            -- quotedTweet.quotedTweet.mentionedUsers.username,
            -- quotedTweet.quotedTweet.quotedTweet.mentionedUsers.username
        FROM `project-latam-challenge.twitter_data.farmers_protest_tweets_2021` 
        LEFT JOIN UNNEST(mentionedUsers) AS mentionedUsers
        WHERE date is not null AND mentionedUsers.username IS NOT NULL
        UNION ALL

        SELECT quotedTweet_mentionedUsers.username AS username
            -- quotedTweet.quotedTweet.mentionedUsers.username,
            -- quotedTweet.quotedTweet.quotedTweet.mentionedUsers.username
        FROM `project-latam-challenge.twitter_data.farmers_protest_tweets_2021` 
        LEFT JOIN UNNEST(quotedTweet.mentionedUsers) AS quotedTweet_mentionedUsers
        WHERE date is not null AND quotedTweet_mentionedUsers.username IS NOT NULL

        UNION ALL

        SELECT quotedTweet_quotedTweet_mentionedUsers.username AS username
            -- quotedTweet.quotedTweet.quotedTweet.mentionedUsers.username
        FROM `project-latam-challenge.twitter_data.farmers_protest_tweets_2021` 
        LEFT JOIN UNNEST(quotedTweet.quotedTweet.mentionedUsers) AS quotedTweet_quotedTweet_mentionedUsers
        WHERE date is not null AND quotedTweet_quotedTweet_mentionedUsers.username IS NOT NULL

        UNION ALL

        SELECT quotedTweet_quotedTweet_quotedTweet_mentionedUsers.username AS username
            -- quotedTweet.quotedTweet.quotedTweet.mentionedUsers.username
        FROM `project-latam-challenge.twitter_data.farmers_protest_tweets_2021` 
        LEFT JOIN UNNEST(quotedTweet.quotedTweet.quotedTweet.mentionedUsers) AS quotedTweet_quotedTweet_quotedTweet_mentionedUsers
        WHERE date is not null AND quotedTweet_quotedTweet_quotedTweet_mentionedUsers.username IS NOT NULL
        )
        SELECT username,
        COUNT(*) AS frequency
        FROM TEMP_DATA_001 A
        GROUP BY ALL
        ORDER BY frequency DESC
        LIMIT 10
        """
    
    results = run_bigquery_query(query)
    return [(row.fecha, row.username) for row in results]
```

In [3]:
import os
import time
from memory_profiler import memory_usage
from q3_time import q3_time
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../credentials/project-latam-challenge-749ce1a96052.json"

# Función para medir el tiempo y la memoria de q2_time
def measure_q1_time(file_path):
    start_time = time.time()
    mem_usage, result = memory_usage((q3_time, (file_path,)), retval=True, max_usage=True)
    end_time = time.time()
    duration = end_time - start_time
    return result, duration, mem_usage

if __name__ == "__main__":
    print("Iniciando proceso q1_time...")
    top_10_users, duration, mem_usage = measure_q1_time(file_path)
    print("Resultados:")
    print(top_10_users)
    print(f"Tiempo de ejecución: {duration} segundos")
    print(f"Uso máximo de memoria: {mem_usage} MiB")

Iniciando proceso q1_time...
Leyendo archivo JSON
Creando bucket y subiendo archivo
Bucket bucket-project-latam-challenge-q3-time created in location US.
File farmers-protest-tweets-2021-2-4.json uploaded to bucket-project-latam-challenge-q3-time/farmers-protest-tweets-2021-2-4.json.
Cargando datos a BigQuery
Starting job 1796d4db-1b8c-4d54-a87d-27d18e14ae1b
Job finished.
Loaded 352221 rows.
Ejecutando query
Resultados:
[('narendramodi', 7875), ('Kisanektamorcha', 6135), ('RakeshTikaitBKU', 5544), ('PMOIndia', 4686), ('GretaThunberg', 3939), ('RahulGandhi', 3756), ('rihanna', 3471), ('DelhiPolice', 3420), ('RaviSinghKA', 3381), ('UNHumanRights', 3180)]
Tiempo de ejecución: 50.49053502082825 segundos
Uso máximo de memoria: 2575.734375 MiB



### CONCLUSIÓN GENERAL
GCP es una herramienta adecuada para la realización de este tipo de tareas, debido a que tiene bueno tiempos de respuesta, pese a un bajo uso de memoria local, esto se logra principalmente gracias a la infraestructura de procesamiento distribuido que utiliza Bigquery.
